# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

81cbbd3775a5494ba18aca4a922f94c3


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,-0.91,98,100,5.16,GL,1734590524
1,1,kirensk,57.7853,108.1119,-19.19,98,25,0.10,RU,1734590525
2,2,shakargarh,32.2623,75.1668,19.48,20,0,2.96,PK,1734590526
3,3,puerto natales,-51.7236,-72.4875,4.56,93,100,8.75,CL,1734590528
4,4,tiksi,71.6872,128.8694,-25.46,98,43,3.10,RU,1734590529


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City"    
)

# Display the map
# YOUR CODE HERE

map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

better_city_df = city_data_df[city_data_df['Cloudiness'] == 0]
better_city_df = better_city_df[better_city_df['Wind Speed'] < 5]
better_city_df = better_city_df[better_city_df['Max Temp'] > 20]


# Drop any rows with null values
# YOUR CODE HERE

better_city_df = better_city_df.dropna()

# Display sample data
# YOUR CODE HERE

better_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,adamstown,-25.0660,-130.1015,21.43,61,0,3.47,PN,1734590539
31,31,lihue,21.9789,-159.3672,23.77,88,0,0.00,US,1734590561
72,72,holualoa,19.6228,-155.9522,21.57,81,0,0.65,US,1734590610
84,84,hermanus,-34.4187,19.2345,22.11,59,0,2.97,ZA,1734590624
89,89,fort abbas,29.1925,72.8536,20.56,15,0,2.48,PK,1734590630


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = better_city_df[["City_ID", "City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotel_df['Hotel Name'] = ""

# Display sample data
# YOUR CODE HERE

hotel_df

C:\Users\David\AppData\Local\Temp\ipykernel_15080\2027135621.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
13,13,adamstown,PN,-25.0660,-130.1015,61,
31,31,lihue,US,21.9789,-159.3672,88,
72,72,holualoa,US,19.6228,-155.9522,81,
84,84,hermanus,ZA,-34.4187,19.2345,59,
89,89,fort abbas,PK,29.1925,72.8536,15,
91,91,pimentel,PE,-6.8367,-79.9342,73,
175,175,shivpuri,IN,25.4333,77.6500,21,
186,186,wailua homesteads,US,22.0669,-159.3780,92,
208,208,chinde,NP,26.8212,87.7095,36,
221,221,kapa'a,US,22.0752,-159.3190,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
categories = "accommodation.hotel"
params = {
    # YOUR CODE HERE
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
holualoa - nearest hotel: Kona Hotel
hermanus - nearest hotel: Aloe guest house
fort abbas - nearest hotel: No hotel found
pimentel - nearest hotel: La Posada
shivpuri - nearest hotel: Hotel SS
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
chinde - nearest hotel: No hotel found
kapa'a - nearest hotel: Pono Kai Resort
aldinga beach - nearest hotel: The Beach Hotel
cooma - nearest hotel: No hotel found
nova vicosa - nearest hotel: Pousada Pontal da Barra
puerto carreno - nearest hotel: Carreño Plaza
kalaoa - nearest hotel: PACIFIC 19 Kona
san juan - nearest hotel: Hotel San Francisco
glen innes - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
kailua-kona - nearest hotel: PACIFIC 19 Kona


,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
13,13,adamstown,PN,-25.0660,-130.1015,61,No hotel found
31,31,lihue,US,21.9789,-159.3672,88,Kauai Palms
72,72,holualoa,US,19.6228,-155.9522,81,Kona Hotel
84,84,hermanus,ZA,-34.4187,19.2345,59,Aloe guest house
89,89,fort abbas,PK,29.1925,72.8536,15,No hotel found
91,91,pimentel,PE,-6.8367,-79.9342,73,La Posada
175,175,shivpuri,IN,25.4333,77.6500,21,Hotel SS
186,186,wailua homesteads,US,22.0669,-159.3780,92,Hilton Garden Inn Kauai Wailua Bay
208,208,chinde,NP,26.8212,87.7095,36,No hotel found
221,221,kapa'a,US,22.0752,-159.3190,92,Pono Kai Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City", 
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)